# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [33]:
from dotenv import load_dotenv
import os
from pathlib import Path
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain, SimpleSequentialChain

raw = Path(".env").read_bytes()
load_dotenv(dotenv_path=Path(".env"), override=True)

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY is not set"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
poem_prompt = ChatPromptTemplate.from_template(
    "다음 프로그래밍 언어에 대한 짧은 시를 영어로 지어줘. 그리고 한글로 번역도 작성 해줘.:\n\n"
    "언어: {language}\n\n"
    "시:"
)

poem_chain = LLMChain(llm=model, prompt=poem_prompt, output_key="poem")

explain_prompt = ChatPromptTemplate.from_template(
    "아래 시를 보고, 콜롬비아 출신의 사람에게 딱 맞는 비유와 은유를 들어 영어로 설명 해줘:\n\n"
    "{poem}\n\n"
    "설명:"
)

explain_chain = LLMChain(llm=model, prompt=explain_prompt, output_key="explanation")

full_chain = poem_chain | explain_chain
full_chain.invoke({"language": "Python"})

# seq_chain = SimpleSequentialChain(
#     chains=[poem_chain, explain_chain],
#     input_key='language',
#     output_key='explanation',
#     verbose=True
# )
#
# result = seq_chain.run({"language": "Python"})

In the world of code, Python slithers with grace,
With its clean syntax and powerful embrace.
From data analysis to web apps so fine,
Python's versatility truly shines.

한글 번역:
코드의 세계에서, 파이썬은 우아하게 기어다닙니다,
깔끔한 구문과 강력한 기능을 갖추고 있죠.
데이터 분석부터 멋진 웹 앱까지,
파이썬의 다재다능함이 찬란히 빛나네요.In the world of code, Python is like a skilled salsa dancer from Colombia, moving with grace and precision. Just like Colombian dancers are known for their fluid movements and elegance, Python is known for its clean syntax and powerful capabilities. Much like how Colombian dancers can seamlessly transition between different styles of dance, Python can be used for a wide range of tasks such as data analysis and creating web applications. Just as Colombian culture is rich and diverse, Python's versatility truly shines in the world of programming.

{'language': 'Python',
 'poem': "In the world of code, Python slithers with grace,\nWith its clean syntax and powerful embrace.\nFrom data analysis to web apps so fine,\nPython's versatility truly shines.\n\n한글 번역:\n코드의 세계에서, 파이썬은 우아하게 기어다닙니다,\n깔끔한 구문과 강력한 기능을 갖추고 있죠.\n데이터 분석부터 멋진 웹 앱까지,\n파이썬의 다재다능함이 찬란히 빛나네요.",
 'explanation': "In the world of code, Python is like a skilled salsa dancer from Colombia, moving with grace and precision. Just like Colombian dancers are known for their fluid movements and elegance, Python is known for its clean syntax and powerful capabilities. Much like how Colombian dancers can seamlessly transition between different styles of dance, Python can be used for a wide range of tasks such as data analysis and creating web applications. Just as Colombian culture is rich and diverse, Python's versatility truly shines in the world of programming."}

# 25.07.01 Challenge

In [42]:
examples = [
    {
        'question': '영화 갓 파더에 대해 설명해줘.',
        'answer': """
            '감독': '프랑시스 포드 코폴라',
            '주요_출연진': ['Marlon Brando', 'Al Pacino', 'James Caan'],
            '예산': 6000000,
            '흥행_수익': 246120974,
            '장르': '범죄',
            '간략한_줄거리': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'
        """
    },
    {
        'question': '오징어 게임에 대해 설명 해줘.',
        'answer': """
            '감독': '황동혁',
            '주요_출연진': ['이정재', '타노스', '이병헌'],
            '예산': 100000,
            '흥행_수익': 423413491,
            '장르': '드라마',
            '간략한_줄거리': '어느날 사람들이 의문의 게임에 초대받고, 생존을 위해 목숨을 건 게임에 참여하게 된다. 이 게임은 상금이 걸린 치열한 경쟁으로, 참가자들은 각자의 사연과 이유로 게임에 참여하게 된다.'
        """
    },
    {
        'question': '인디아나 존스 영화에 대해서 설명 해줘.',
        'answer': """
            '감독': '스티븐 스필버그',
            '주요_출연진': ['Harrison Ford', 'Karen Allen', 'Paul Freeman'],
            '예산': 18000000,
            '흥행_수익': 389925971,
            '장르': '어드벤처',
            '간략한_줄거리': 'An archaeologist is hired by the U.S. government to find the Ark of the Covenant.'
        """
    },
    {
        'question': '영화 타이타닉에 대해서 설명 해줘.',
        'answer': """
            '감독': '제임스 카메론',
            '주요_출연진': ['Kate Winslet', 'Leonardo DiCaprio', 'Billy Zane'],
            '예산': 200000000,
            '흥행_수익': 2187463944,
            '장르': '로맨스',
            '간략한_줄거리': 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'
        """
    }
]

In [48]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)
# print(example_prompt.format(**examples[0]))

In [49]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
)
# print(prompt.format(input="오징어 게임"))

In [50]:
chain = LLMChain(llm=model, prompt=prompt, output_key="answer")

chain.invoke({"input": "마더"})

'감독': '봉준호',
'주요_출연진': ['김혜자', '원빈', '진구'],
'예산': 5000000,
'흥행_수익': 16917651,
'장르': '스릴러',
'간략한_줄거리': '실종된 딸을 찾기 위해 모든 것을 걸고 싸우는 엄마의 이야기를 그린 영화.'

{'input': '마더',
 'answer': "'감독': '봉준호',\n'주요_출연진': ['김혜자', '원빈', '진구'],\n'예산': 5000000,\n'흥행_수익': 16917651,\n'장르': '스릴러',\n'간략한_줄거리': '실종된 딸을 찾기 위해 모든 것을 걸고 싸우는 엄마의 이야기를 그린 영화.'"}